# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
from cassandra.cluster import Cluster
import os
import glob
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/bastiaan/Documents/Projects/sparkify-cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance on your local machine (IP: 127.0.0.1)
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
create_keyspace_query = """CREATE KEYSPACE IF NOT EXISTS sparkify WITH 
REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""

session.execute(create_keyspace_query)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1 data model

The first query is in the form:

**Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4**

A fitting select query meeting the above given query definition is given below:


In [8]:
# Formulate query 1
select_query1 = """SELECT artist, song, length FROM played_songs WHERE 
session_id = '338' AND item_in_session = '4'"""

The data in the Apache Cassandra database is modelled to fit the type of 
queries that are run against the database. For this reason, each query type 
needs its own table to perform the queries on. To be descriptive of the type of 
query performed, note that the table is given the name *played_songs*.

To successfully run the first query, a table must be created that can filer 
based on *session_id* and *item_in_session*. As such, both *session_id* and 
*item_in_session* must be contained in one, compound primary key. This primary 
key also meets the requirement of uniqueness. Moreover, the columns *artist*, 
*song* and *length* that are in the select statement must be included as well.

In [9]:
# Formulate the corresponding table format
create_table_query1 = """CREATE TABLE IF NOT EXISTS played_songs (session_id 
TEXT, item_in_session TEXT, artist TEXT, song TEXT, length TEXT, PRIMARY KEY (
session_id, item_in_session))"""

# Create the table
session.execute(create_table_query1)
                    

In [10]:
# Insert selected data from the csv file into the cassandra database table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO played_songs (session_id, item_in_session, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[8], line[3], line[0], line[9], line[5]))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
# Run the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(select_query1)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.length)


Faithless Music Matters (Mark Knight Dub) 495.3073


#### Query 2 data model

The second query is in the form:

**Query 2:  Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

A fitting select query meeting the above given query definition is given below:

In [12]:
# Formulate query 2
select_query2 = """SELECT artist, song, first_name, last_name FROM user_plays 
WHERE user_id = '10' AND session_id = '182'"""

Again, this query needs a separated table to be queried. To give it a name descriptive of the queries performed upon it, the table is named *user_plays*.

Running the second query requires a table that can filter based on *user_id* 
and *session_id*. Moreover, the results should be ordered based on 
*item_in_session*. To make this possible, and make the primary key unique as 
well, the primary key must comprise *user_id*, *session_id* and 
*item_in_session*. The columns *user_id* and *session_id* may hereby act as 
compound partition key, where *item_in_session* then constitutes a clustering 
column. The columns *artist*, *song*, *first_name* and *last_name* must be 
included as well, as they are in the select statement.

In [13]:
# Formulate the corresponding table format
create_table_query2 = """CREATE TABLE IF NOT EXISTS user_plays (user_id 
TEXT, session_id TEXT, item_in_session TEXT, artist TEXT, song TEXT, 
first_name TEXT, last_name TEXT, PRIMARY KEY ((user_id, session_id), 
item_in_session))"""

# Create the table
session.execute(create_table_query2)
                    

In [14]:
# Insert selected data from the csv file into the cassandra database table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_plays (user_id, session_id, item_in_session, artist, song, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[10], line[8], line[3], line[0], line[9], line[1], line[4]))
                    

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
# Run the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(select_query2)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)


Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Query 3 data model

The third query is in the form:

**Query 2:  Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

A fitting select query meeting the above given query definition is given below:

In [16]:
# Formulate query 3
select_query3 = """SELECT first_name, last_name FROM song_listeners WHERE song = 
'All Hands Against His Own'"""

Again, this query needs a separated table to be queried. To give it a name descriptive of the queries performed upon it, the table is named *song_listeners*.

The third query requires a table that can filter based on *song*
(song title). To make the primary key unique, an efficient choice would be to 
include *song* together with *user_id* in the primary key. For the partition 
key, *song* is chosen. Finally, the columns *first_name* and *last_name* are 
included as they are included in the select statement.

In [18]:
# Formulate the corresponding table format
create_table_query3 = """CREATE TABLE IF NOT EXISTS song_listeners (song TEXT, 
user_id TEXT, first_name TEXT, last_name TEXT, PRIMARY 
KEY (song, user_id))"""

# Create the table
session.execute(create_table_query3)

In [21]:
# Insert selected data from the csv file into the cassandra database table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_listeners (song, user_id, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], line[10], line[1], line[4]))
        

#### Do a SELECT to verify that the data have been inserted into each table

In [22]:
# Run the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(select_query3)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [23]:
# Drop the tables before closing out the session
session.execute("DROP TABLE IF EXISTS played_songs")
session.execute("DROP TABLE IF EXISTS user_plays")
session.execute("DROP TABLE IF EXISTS song_listeners")

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()